# Importing Libraries

In [1]:
import sklearn
print(sklearn.__version__)
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy.lib.function_base import vectorize
from sklearn.feature_extraction.text import CountVectorizer
from numpy.core.fromnumeric import size
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import tqdm

1.2.2


# Data-Set

In [28]:
# データの読み込み
df_2007 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2007.csv", header=0)
df_2008 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2008.csv", header=0)
df_2009 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2009.csv", header=0)
df_2010 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2010.csv", header=0)
df_2011 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2011.csv", header=0)
df_2012 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2012.csv", header=0)
df_2013 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2013.csv", header=0)
df_2014 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2014.csv", header=0)
df_2015 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2015.csv", header=0)
df_2016 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2016.csv", header=0)
df_2017 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2017.csv", header=0)
df_2018 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2018.csv", header=0)
df_2019 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test/data_set_2019.csv", header=0)

# df_0000 storage list
list_df = [df_2007, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019]

# Reading and data splitting

In [36]:
def DataMake(split, metric, list_df):
  # variable
  label_train = []
  label_test = []
  sentences_train = []
  sentences_test = []

  # Split the correct answer labels and target sentences into study and test.
  list_train = list_df[:split]
  list_test = list_df[split:]
  print('metric:', metric)
  print('train:', len(list_train), 'test:', len(list_test))

  # Store data frames for testing in df_test.
  df_test = pd.concat(list_test)

  # Extract and list metric value in a list.
  for i in list_train:
    label_train.append(i[metric].values)
    sentences_train.append(i['Description'].values)
  for i in list_test:
    label_test.append(i[metric].values)
    sentences_test.append(i['Description'].values)

  # data for input
  # metric value
  y_train = np.concatenate(label_train, 0)
  y_test = np.concatenate(label_test, 0)
  # description
  train_sentence = np.concatenate(sentences_train, 0)
  test_sentence = np.concatenate(sentences_test, 0)

  return y_train, y_test, train_sentence, test_sentence, df_test

# Natural Language Processing (train)

In [30]:
def NLP_train(train_sentence):
  # Natural Language Processing
  # Creating BoW features with sklearn's CountVectorizer
  vectorizer = CountVectorizer(stop_words="english")
  X_train = vectorizer.fit_transform(train_sentence)
  
  return X_train, vectorizer

# Multinomial Logistic Regression (train)

In [32]:
def MLR_train(X_train, y_train):
  # Create a classification model for MLR using vectorized features
  lr = LogisticRegression(C=0.3, random_state=0, n_jobs=-1)
  lr.fit(X_train, y_train)

  return lr



# Natural Language Processing (test)

In [31]:
def NLP_test(test_sentence, vectorizer):
  # Natural Language Processing
  # Creating BoW features with sklearn's CountVectorizer.
  X_test = vectorizer.transform(test_sentence)
  
  return X_test

# Multinomial Logistic Regression (test)

In [33]:
def MLR_test(metric, X_test, y_test, lr):
  # Test data to confirm accuracy.
  y_pred = lr.predict(X_test)
  cm = confusion_matrix(y_test, y_pred)

  # Branching when creating a table.
  if metric == 'AV':
    table = pd.DataFrame(cm, columns=['Predicted P', 'Predicted L', 'Predicted A', 'Predicted N'], index=['Actual P', 'Actual L', 'Actual A', 'Actual N'])
  elif metric == 'AC':
    table = pd.DataFrame(cm, columns=['Predicted L', 'Predicted H'], index=['Actual L', 'Actual H'])
  elif metric == 'PR':
    table = pd.DataFrame(cm, columns=['Predicted N', 'Predicted L', 'Predicted H'], index=['Actual N', 'Actual L', 'Actual H'])
  elif metric == 'UI':
    table = pd.DataFrame(cm, columns=['Predicted N', 'Predicted R'], index=['Actual N', 'Actual R'])
  elif metric == 'S':
    table = pd.DataFrame(cm, columns=['Predicted U', 'Predicted C'], index=['Actual U', 'Actual C'])
  else:
    table = pd.DataFrame(cm, columns=['Predicted N', 'Predicted L', 'Predicted H'], index=['Actual N', 'Actual L', 'Actual H'])

  # Accuracy
  print("accuracy:", accuracy_score(y_test, y_pred))
  print(table)
  print('-'*70)

  # Returns a list containing the prediction results and a data frame for testing.
  return y_pred, df_test



# Example

In [37]:
metrics = ['AV', 'AC', 'PR', 'UI', 'S', 'C', 'I', 'A']
split = 11

for metric in metrics:
  y_train, y_test, train_sentence, test_sentence, df_test = DataMake(split, metric, list_df)

  X_train, vectorizer = NLP_train(train_sentence)
  lr = MLR_train(X_train, y_train)

  X_test = NLP_test(test_sentence, vectorizer)
  MLR_test(metric, X_test, y_test, lr)

metric: AV
train: 11 test: 1
accuracy: 0.8711995159582514
          Predicted P  Predicted L  Predicted A  Predicted N
Actual P           62           23          166            0
Actual L            9         1939          731            2
Actual A           45          616         9462            4
Actual N            1           35           71           56
----------------------------------------------------------------------
metric: AC
train: 11 test: 1
accuracy: 0.9461503554681592
          Predicted L  Predicted H
Actual L          425          358
Actual H          354        12085
----------------------------------------------------------------------
metric: PR
train: 11 test: 1
accuracy: 0.7798366359098472
          Predicted N  Predicted L  Predicted H
Actual N          272          308          325
Actual L           85         1986         1527
Actual H           43          623         8053
----------------------------------------------------------------------
metric: UI
